<a href="https://colab.research.google.com/github/olegkhomenko/pixelate-me-tg/blob/master/pixelate_me_tg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install gdown > /dev/null
!pip3 install git+https://github.com/sedthh/pyxelate.git > /dev/null
!pip3 install face_recognition > /dev/null
!git clone https://github.com/Charleshhy/Grapy-ML > /dev/null

# We need 'GPM-ML_finetune_PASCAL.pth' file from
# https://drive.google.com/drive/folders/1eQ9IV4QmcM5dLCuVMSVE3ogVpL6qUQL5
#
# Let's download it directly using gdown and unique 'data-id'
!gdown https://drive.google.com/uc?id=1QPwUY9vWmDMnuey2RK4Ief2u4CHqyRjZ > /dev/null

import os
import sys

sys.path.insert(0, f'{os.getcwd()}/Grapy-ML/')

In [0]:
from typing import Optional, Union

import numpy as np
import torch
import torch.nn.functional as F
from torchvision import transforms
from torchvision.utils import make_grid

import face_recognition
from networks import grapy_net
from pyxelate import Pyxelate
from utils.util import decode_seg_map_sequence


def box_size(box: tuple):
    return (box[2] - box[0]) * (box[1] - box[3])


def largest_face(face_locations: list):
    idx = np.argmax([box_size(box) for box in face_locations])
    return face_locations[idx]


def get_cropped_image(image: np.ndarray, face_padding=0.6):
    face_locations = face_recognition.face_locations(image)

    if not len(face_locations):
        return None

    face_coord = largest_face(face_locations)

    h, w, ch = image.shape
    box_h = face_coord[2] - face_coord[0]
    box_w = face_coord[1] - face_coord[3]

    left = max(0, int(face_coord[3] - box_w * face_padding))
    right = min(w, int(face_coord[1] + box_w * face_padding))
    top = min(h, int(face_coord[2] + box_h * face_padding))
    bot = max(0, int(face_coord[0] - box_h * face_padding))

    return image[bot:top, left:right]


def get_segm_net(weights_path='/content/GPM-ML_finetune_PASCAL.pth'):
    net = grapy_net.GrapyMutualLearning(os=16, hidden_layers=256)
    net.load_state_dict(torch.load(weights_path))

    c1, c2, p1, p2, a1, a2 = (
        [[0], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]],
        [[0], [1, 2, 4, 13], [5, 6, 7, 10, 11, 12], [3, 14, 15], [8, 9, 16, 17, 18, 19]],
        [[0], [1, 2, 3, 4, 5, 6]],
        [[0], [1], [2], [3, 4], [5, 6]],
        [[0], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]],
        [[0], [1, 2, 3, 11], [4, 5, 7, 8, 16, 17], [14, 15], [6, 9, 10, 12, 13]]
    )

    net.set_category_list(c1, c2, p1, p2, a1, a2)
    net.eval()
    net.cuda()
    return net


def pixelate(image: Union[str, np.ndarray],
             colors=10, factor=8, dither=False, quantize_by_factor=4,
             segmentation_network=None) -> Optional[dict]:
    assert isinstance(image, (str, np.ndarray))

    if isinstance(image, str):
        image = face_recognition.load_image_file(image)

    cropped_image = get_cropped_image(image)
    if cropped_image is None:
        return None

    img = cropped_image

    result = {}

    height, width, _ = img.shape

    p = Pyxelate(height // factor, width // factor, colors, dither)
    img_small = p.convert(((img // quantize_by_factor) * quantize_by_factor).astype('uint8'))

    result['img_small'] = img_small
    result['cropped_image'] = cropped_image

    if segmentation_network:
        segm_net = segmentation_network

        t = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((.5, ), (.5, )),
                                lambda x: x.unsqueeze(0).cuda(),
                                ])

        # Getting segmentation mask
        with torch.no_grad():
            outputs, *_ = segm_net((t(img), 1), training=False)

        # Decode segmentation mask
        grid_image = make_grid(
            decode_seg_map_sequence(torch.max(outputs[:3], 1)[1].detach().cpu().numpy()), 3,
            normalize=False,
            range=(0, 255))

        # Mask binarization
        mask_tens = (grid_image.sum(axis=0, keepdim=True) > 0).unsqueeze(0).float()
        mask = mask_tens.cpu().squeeze(0).permute(1, 2, 0).numpy()

        # Finding edges
        conv_filter = torch.ones(1, 1, 3, 3)
        mask_tens = F.interpolate(mask_tens, (height // factor, width // factor))
        contours = F.conv2d(mask_tens, conv_filter, padding=1)
        contours = ((contours[0] > 1) & (contours[0] < 6)).float()
        contours = contours.permute(1, 2, 0).numpy()

        masked_image = img * mask.astype('uint8') + ((mask.repeat(3, axis=2) - 1) * -254).astype('uint8')

        img = masked_image

        img_segm_small = p.convert((((img // quantize_by_factor)) * quantize_by_factor + (0.75 * quantize_by_factor)).astype('uint8'))
        result['img_segm_small'] = img_segm_small

        img_small_w_contour = (img_small - (contours * 255)).clip(0, 255).astype('uint8')
        result['img_small_w_contour'] = img_small_w_contour

        img_segm_small_w_contour = (img_segm_small - (contours * 255)).clip(0, 255).astype('uint8')
        result['img_segm_small_w_contour'] = img_segm_small_w_contour

    return result

# Telegram BOT

In [0]:
!pip3 install requests > /dev/null
!pip3 install pyTelegramBotAPI > /dev/null

import matplotlib.pyplot as plt
import logging
from io import BytesIO

import numpy as np
import requests
from PIL import Image
from skimage.transform import rescale, resize

import telebot

# SET YOU TELEGRAM TOKEN FROM @BotFather BELOW
TOKEN = None
assert TOKEN is not None

In [0]:
bot = telebot.TeleBot(TOKEN)
telebot.logger.setLevel(logging.DEBUG)
segm_net = get_segm_net()

@bot.message_handler(commands=['hi'])
def send_welcome(message):
    bot.reply_to(message, 'Hi! You can send me a photo')


@bot.message_handler(func=lambda message: True)
def echo_message(message):
    bot.reply_to(message, message.text)


@bot.message_handler(content_types= ["photo"])
def pixelate_handler(message):
    global img
    bot.send_message(message.chat.id, "Got your photo. Start working..")
    # Getting photo
    file_info = bot.get_file(message.photo[-1].file_id)
    r = requests.get('https://api.telegram.org/file/bot{0}/{1}'.format(TOKEN, file_info.file_path))
    img = np.array(Image.open(BytesIO(r.content)))
    # Pixelating
    res = pixelate(img, segmentation_network=segm_net)
    if res is not None:
        res = res['img_segm_small_w_contour']
        # Resizing to ~512
        res_rescaled = rescale(res, (int(512 / res.shape[0]), int(512 / res.shape[0]), 1), anti_aliasing=False, order=0)
        # Sending back
        buf = BytesIO()
        plt.imsave(buf, res_rescaled, format='jpg')
        bot.send_photo(message.chat.id, buf.getvalue())
    else:
        bot.send_message(message.chat.id, 'No faces detected. Please look at camera')

bot.polling()